# Session 16 - EDA Complète du Titanic

## 🎯 Objectifs
- Exploration visuelle approfondie
- Analyse des corrélations
- Répondre à des questions métier complexes
- Identifier des patterns et insights

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print('✓ Bibliothèques chargées')

## Partie 1 : Chargement et vue d'ensemble

In [ ]:
# Charger les données nettoyées
df = pd.read_csv('../data/titanic_clean.csv')

print(f'Dataset : {df.shape[0]} lignes x {df.shape[1]} colonnes')
print(f'Taux de survie global : {df["Survived"].mean():.2%}\n')
df.head()

In [ ]:
# Informations générales
print('=== INFORMATIONS ===' )
df.info()

In [ ]:
# Statistiques descriptives
print('=== STATISTIQUES DESCRIPTIVES ===')
df.describe()

## Partie 2 : Analyse univariée - Variables numériques

In [ ]:
# Distribution de l'âge
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

axes[0].hist(df['Age'], bins=30, edgecolor='black', alpha=0.7, color='skyblue')
axes[0].set_title('Distribution de l\'âge', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Âge')
axes[0].set_ylabel('Fréquence')
axes[0].axvline(df['Age'].mean(), color='red', linestyle='--', label=f'Moyenne: {df["Age"].mean():.1f}')
axes[0].axvline(df['Age'].median(), color='green', linestyle='--', label=f'Médiane: {df["Age"].median():.1f}')
axes[0].legend()

axes[1].boxplot(df['Age'])
axes[1].set_title('Boxplot de l\'âge', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Âge')

sns.kdeplot(data=df, x='Age', ax=axes[2], fill=True, color='purple')
axes[2].set_title('Densité de l\'âge', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Âge')

plt.tight_layout()
plt.show()

print(f'📊 Âge moyen : {df["Age"].mean():.1f} ans')
print(f'📊 Âge médian : {df["Age"].median():.1f} ans')
print(f'📊 Écart-type : {df["Age"].std():.1f} ans')

In [ ]:
# Distribution du prix des billets
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df['Fare'], bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[0].set_title('Distribution des prix (bruts)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Prix (£)')
axes[0].set_ylabel('Fréquence')

# Log scale pour mieux voir
axes[1].hist(np.log1p(df['Fare']), bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[1].set_title('Distribution des prix (échelle log)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Log(Prix + 1)')
axes[1].set_ylabel('Fréquence')

plt.tight_layout()
plt.show()

print(f'📊 Prix moyen : £{df["Fare"].mean():.2f}')
print(f'📊 Prix médian : £{df["Fare"].median():.2f}')
print(f'📊 Prix minimum : £{df["Fare"].min():.2f}')
print(f'📊 Prix maximum : £{df["Fare"].max():.2f}')

## Partie 3 : Analyse univariée - Variables catégorielles

In [ ]:
# Distribution par classe
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Countplot
sns.countplot(data=df, x='Pclass', ax=axes[0], palette='Set2')
axes[0].set_title('Nombre de passagers par classe', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Classe')
axes[0].set_ylabel('Nombre de passagers')
for container in axes[0].containers:
    axes[0].bar_label(container)

# Pie chart
df['Pclass'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.1f%%', 
                                  colors=['#FF6B6B', '#4ECDC4', '#95E1D3'])
axes[1].set_title('Répartition par classe', fontsize=14, fontweight='bold')
axes[1].set_ylabel('')

# Distribution par sexe
sns.countplot(data=df, x='Sex', ax=axes[2], palette='pastel')
axes[2].set_title('Nombre de passagers par sexe', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Sexe')
for container in axes[2].containers:
    axes[2].bar_label(container)

plt.tight_layout()
plt.show()

## Partie 4 : Analyse bivariée - Survie vs autres variables

In [ ]:
# Question 1 : Taux de survie par sexe et classe
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Par sexe
survival_sex = df.groupby('Sex')['Survived'].mean()
survival_sex.plot(kind='bar', ax=axes[0], color=['#FF6B6B', '#4ECDC4'])
axes[0].set_title('Taux de survie par sexe', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Taux de survie')
axes[0].set_xlabel('Sexe')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)
for i, v in enumerate(survival_sex):
    axes[0].text(i, v + 0.02, f'{v:.2%}', ha='center', fontweight='bold')

# Par classe
survival_class = df.groupby('Pclass')['Survived'].mean()
survival_class.plot(kind='bar', ax=axes[1], color=['#95E1D3', '#FECA57', '#FF6B6B'])
axes[1].set_title('Taux de survie par classe', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Taux de survie')
axes[1].set_xlabel('Classe')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)
for i, v in enumerate(survival_class):
    axes[1].text(i, v + 0.02, f'{v:.2%}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print('💡 INSIGHT : Les femmes ont ~4x plus de chances de survivre que les hommes')
print('💡 INSIGHT : La 1ère classe a 2.5x plus de chances que la 3ème classe')

In [ ]:
# Heatmap : Taux de survie par classe ET sexe
survival_crosstab = df.groupby(['Pclass', 'Sex'])['Survived'].mean().unstack()

plt.figure(figsize=(8, 6))
sns.heatmap(survival_crosstab, annot=True, fmt='.2%', cmap='RdYlGn', 
            cbar_kws={'label': 'Taux de survie'}, linewidths=2)
plt.title('Taux de survie par classe et sexe', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Sexe', fontsize=12)
plt.ylabel('Classe', fontsize=12)
plt.show()

print('\n💡 INSIGHT : Les femmes de 1ère et 2ème classe ont un taux de survie > 90%')
print('💡 INSIGHT : Les hommes de 3ème classe ont le taux le plus bas (~13%)') 

In [ ]:
# Question 2 : Impact de l'âge sur la survie
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribution d'âge selon survie
sns.histplot(data=df, x='Age', hue='Survived', bins=30, kde=True, ax=axes[0], palette='Set1')
axes[0].set_title('Distribution de l\'âge selon la survie', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Âge')
axes[0].legend(['Non-survivant', 'Survivant'])

# Boxplot
sns.boxplot(data=df, x='Survived', y='Age', ax=axes[1], palette='Set2')
axes[1].set_title('Âge selon la survie (boxplot)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Survie (0=Non, 1=Oui)')
axes[1].set_ylabel('Âge')

plt.tight_layout()
plt.show()

print(f'Âge moyen des survivants : {df[df["Survived"]==1]["Age"].mean():.1f} ans')
print(f'Âge moyen des non-survivants : {df[df["Survived"]==0]["Age"].mean():.1f} ans')
print('\n💡 INSIGHT : Les enfants (< 10 ans) ont un meilleur taux de survie')

## Partie 5 : Analyse des corrélations

In [ ]:
# Matrice de corrélation
numeric_cols = ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FamilySize', 'IsAlone']
corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={'shrink': 0.8})
plt.title('Matrice de corrélation', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print('📊 Corrélations avec la survie :')
print(corr_matrix['Survived'].sort_values(ascending=False))

In [ ]:
# Top corrélations (en valeur absolue)
corr_with_survival = corr_matrix['Survived'].drop('Survived').abs().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
corr_with_survival.plot(kind='barh', color='teal')
plt.title('Variables les plus corrélées avec la survie (valeur absolue)', 
          fontsize=14, fontweight='bold')
plt.xlabel('Corrélation absolue')
plt.ylabel('Variable')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print('\n💡 INSIGHT : Pclass, Sex et Fare sont les facteurs les plus corrélés avec la survie')

## Partie 6 : Questions métier complexes

In [ ]:
# Question : Impact de la taille de la famille sur la survie
family_survival = df.groupby('FamilySize').agg({
    'Survived': ['mean', 'count']
}).round(3)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Taux de survie
df.groupby('FamilySize')['Survived'].mean().plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Taux de survie selon taille de famille', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Taille de la famille')
axes[0].set_ylabel('Taux de survie')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)
axes[0].axhline(df['Survived'].mean(), color='red', linestyle='--', 
                 label=f'Moyenne globale: {df["Survived"].mean():.2%}')
axes[0].legend()

# Nombre de passagers
df.groupby('FamilySize').size().plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Nombre de passagers par taille de famille', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Taille de la famille')
axes[1].set_ylabel('Nombre de passagers')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)

plt.tight_layout()
plt.show()

print(family_survival)
print('\n💡 INSIGHT : Les familles de 2-4 personnes ont le meilleur taux de survie')
print('💡 INSIGHT : Les passagers seuls et les très grandes familles ont un taux plus faible')

In [ ]:
# Question : Prix du billet selon classe et survie
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.boxplot(data=df, x='Pclass', y='Fare', hue='Survived', ax=axes[0], palette='Set2')
axes[0].set_title('Prix selon classe et survie', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Classe')
axes[0].set_ylabel('Prix (£)')
axes[0].legend(title='Survie', labels=['Non', 'Oui'])

sns.violinplot(data=df, x='Pclass', y='Fare', hue='Survived', 
               split=True, ax=axes[1], palette='muted')
axes[1].set_title('Distribution des prix (violin plot)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Classe')
axes[1].set_ylabel('Prix (£)')
axes[1].legend(title='Survie', labels=['Non', 'Oui'])

plt.tight_layout()
plt.show()

print('💡 INSIGHT : Dans chaque classe, les survivants ont payé légèrement plus cher en moyenne')

## Partie 7 : Insights et conclusions

### 🎯 Facteurs clés de survie identifiés :

1. **Sexe** : Facteur le plus important
   - Femmes : 74% de survie
   - Hommes : 19% de survie
   - Principe 'femmes et enfants d'abord' clairement appliqué

2. **Classe sociale** : Impact majeur
   - 1ère classe : 63% de survie
   - 2ème classe : 47% de survie  
   - 3ème classe : 24% de survie
   - Accès privilégié aux canots de sauvetage

3. **Âge** : Les enfants prioritaires
   - Enfants (< 10 ans) : Taux de survie élevé
   - Adultes jeunes : Taux modéré
   - Personnes âgées : Taux plus faible

4. **Taille de la famille** : Effet non-linéaire
   - Familles moyennes (2-4) : Meilleur taux
   - Seuls ou grandes familles : Taux plus faible
   - Entraide vs coordination difficile

5. **Prix du billet** : Corrélé à la classe
   - Prix élevé = meilleur emplacement
   - Accès plus rapide aux ponts supérieurs

### 🔍 Profils types :

**Profil à haut risque** :
- Homme adulte de 3ème classe, voyageant seul
- Taux de survie : ~10-15%

**Profil à bas risque** :
- Femme ou enfant de 1ère classe, en famille
- Taux de survie : ~90-95%

### 📊 Recommandations pour modélisation :

1. Utiliser Sex, Pclass et Age comme features principales
2. Créer des interactions : Sex × Pclass, Age × Sex
3. Considérer FamilySize comme feature non-linéaire
4. Normaliser Fare (échelle log)
5. Encoder les variables catégorielles (Sex, Embarked, Title)